In [2]:
import json

import pandas as pd
import sqlalchemy as sa
import facility_z_score as fz

from datetime import datetime

In [3]:
engine = fz.get_sqlalchemy_engine()
facility_data = fz.get_facility_data(engine)
facility_data.sort_values(by=['Year','Month'], inplace=True, ascending=False)

new_facility_data = fz.transform_facility_data(facility_data)
print(new_facility_data.columns[1])
new_facility_data['Average'] = new_facility_data.iloc[:, 2:].mean(axis=1)
new_facility_data['Min'] = new_facility_data.iloc[:, 2:].min(axis=1)
new_facility_data['Max'] = new_facility_data.iloc[:, 2:].max(axis=1)
new_facility_data['Stdev'] = new_facility_data.iloc[:, 2:].std(axis=1)
new_facility_data['Value - Average'] = new_facility_data.iloc[:, 1] - new_facility_data['Average']
# Calculate z score for the 3rd columns (2nd index)
new_facility_data['Z-Score'] = new_facility_data['Value - Average']/new_facility_data['Stdev']
new_facility_data['Median'] = new_facility_data.iloc[:, 2:].median(axis=1)
new_facility_data['Average/Median'] = new_facility_data['Average']/ new_facility_data['Median']

# absoulte value of z score
new_facility_data['Z-Score_abs'] = new_facility_data['Z-Score'].abs()
new_facility_data.sort_values(by=['Z-Score_abs'], inplace=True, ascending=False)
new_facility_data.drop(columns=['Z-Score_abs'], inplace=True)
new_facility_data[['Average','Min','Max','Stdev','Value - Average','Z-Score','Median', 'Average/Median',"FacilityID"]]

rearrange_df = new_facility_data[['Average','Min','Max','Stdev','Value - Average','Z-Score','Median', 'Average/Median',"FacilityID"]].copy()

for col in new_facility_data.columns:
    if col not in rearrange_df.columns:
        rearrange_df[col] = new_facility_data[col]

rearrange_df.to_csv("most_recent_month_facility_z_score.csv", index=False)

flagged_data = fz.check_spending_variations(rearrange_df)
IQR_and_flagged_data = fz.identify_outliers_iqr(flagged_data)

IQR_and_flagged_data.to_csv("facility_outliers.csv", index=False)


Jan-24
['Jan-24', 'Dec-23', 'Nov-23', 'Oct-23', 'Sep-23', 'Aug-23', 'Jul-23', 'Jun-23', 'May-23', 'Apr-23', 'Mar-23', 'Feb-23', 'Jan-23', 'Dec-22', 'Nov-22', 'Oct-22', 'Sep-22', 'Aug-22', 'Jul-22', 'Jun-22', 'May-22', 'Apr-22', 'Mar-22', 'Feb-22', 'Jan-22']
3840 Jan-24 441263.88
3840 Dec-23 607346.37
11076 Jan-24 7680885.527
11076 Aug-22 6354063.9135
139 Feb-22 3758383.8974
139 Jan-22 3581372.7298


KeyboardInterrupt: 